In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 14.4 MB/s 


In [3]:
import numpy as np 
import pandas as pd 
import cv2
from PIL import Image 
import pathlib
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras as keras
from tensorflow.keras.applications.resnet import ResNet152
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout

In [4]:
pathToTrainData='/content/drive/MyDrive/cs331FinalProject/Experiment/split2/train/'
pathToTestData = '/content/drive/MyDrive/cs331FinalProject/Experiment/split2/test/'
num_classes = 20

# Part 1 - Data Preprocessing

# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory(pathToTrainData,
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 shuffle = True,
                                                 class_mode = 'categorical')

# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(pathToTestData,
                                            target_size = (224, 224))

Found 5081 images belonging to 20 classes.
Found 1260 images belonging to 20 classes.


In [5]:
resnet = ResNet152(include_top=False, weights='imagenet', input_shape=(224, 244, 3),pooling='max')
output = resnet.layers[-1].output
output = tf.keras.layers.Flatten()(output)
resnet = Model(resnet.input, output)

for layer in resnet.layers:
    layer.trainable = False

resnet.summary()

234698864/234698864 [==============================] - 7s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 244, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 250, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 122, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                           

In [6]:
model = Sequential()
model.add(resnet)
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 2048)              58370944  
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_2 (Dense)             (None, 20)                20500     
                                                                 
Total params: 61,539,220
Trainable params: 3,168,276
Non

In [7]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8,
                                              restore_best_weights=False)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                   factor=0.2,
                                   patience=4,
                                   verbose=1,
                                   min_delta=5*1e-3,min_lr = 5*1e-7,
                                   )

In [8]:
model.compile(optimizer = adam, loss = 'categorical_crossentropy', 
              metrics=[tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),'accuracy'])

In [9]:
step_reduction = True

if step_reduction:
    history = model.fit(training_set,
        steps_per_epoch=np.ceil(float(18522) / float(32)),
        epochs=20,
        validation_steps=np.ceil(float(2058) / float(32)),
        validation_data=test_set)
else:
    history = model.fit(training_set,
          epochs=20,
          validation_data=test_set)
    
model.evaluate(test_set)

Epoch 1/20
159/579 [=======>......................] - ETA: 1:37:12 - loss: 3.4184 - categorical_crossentropy: 3.4185 - accuracy: 0.0781

40/40 [==============================] - 17s 409ms/step - loss: 2.8534 - categorical_crossentropy: 2.8543 - accuracy: 0.1460


[2.8533971309661865, 2.8543295860290527, 0.14603175222873688]